<div style="text-align: justify">
    
# **Updated population density of the LA city neighborhood councils.**
    
### Here is a notebook to evaluate the populataion, area(in square miles) and population density of the 99 Neighborhood councils (NCs). 
   
**Have calculated the population of each neighborhood council using the [census tract 2020](https://www2.census.gov/geo/tiger/TIGER2020PL/STATE/06_CALIFORNIA/06037/), [updated NC shape file](https://geohub.lacity.org/datasets/9c8639737e3a457a8c0f6a93f9c36974_18/about)- 99 councils and the American Community Survey (ACS) 2020 demographics data.**

**Note: The Tiger zip file that was downloaded- tl_2020_06037_tract20.zip.** 

**Save the ACS 2020 demographics data from this link-https://api.census.gov/data/2020/acs/acs5/profile?get=group(DP05)&for=tract:*&in=state:06&in=county:037&key=cb51343974c4b01dc140a03af63a82c6424272ee) as a csv file and clean it (removing the empty space, closed bracket and quotes). Use this [resource](https://api.census.gov/data/2020/acs/acs5/profile/variables) to understand the variables in the ACS file. I found this [video](https://www.census.gov/library/video/2020/using-api-all-results-for-acs-table.html) very useful.**    

In [1]:
# Importing the necessary packages.
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt 
import folium
from shapely.geometry import Point, Polygon 
from pyproj import Geod
import webbrowser

In [2]:
os.getcwd()
os.chdir('C:/Users/AdithiPriya/Desktop/Hack for LA/Geospatial analysis')

# Census 2020 data.
census_2020 = gpd.read_file('C:/Users/AdithiPriya/Desktop/Hack for LA/Geospatial analysis/tl_2020_06037_tract20/tl_2020_06037_tract20.shp')

# LA NC shape file.
la_nc = gpd.read_file('C:/Users/AdithiPriya/Desktop/Hack for LA/Geospatial analysis/Neighborhood_Councils_(Certified)/Neighborhood_Councils_(Certified).shp')

# ACS demographics data.
acs_demo = pd.read_csv('ACS_census_tract_LA.csv')
acs_demo['GEO_ID']= acs_demo['GEO_ID'].str.replace('1400000US','')

# rename GEO_ID as GEOID20.

acs_demo= acs_demo.rename(columns={'GEO_ID':'GEOID20', 'Total population': 'population'})

### Defining a [function](https://github.com/hackforla/access-the-data-workshop-311-analysis/blob/main/notebooks/NC-population-density.ipynb) to compute the area in square miles (area (sq_miles)).

In [3]:
geod = Geod(ellps= 'WGS84') 

def area_sq_miles(geo):
    area_sq_meters= abs(geod.geometry_area_perimeter(geo)[0])
    return (area_sq_meters * 3.86102e-7)

In [4]:
# Taking a subset of the acs_demo dataframe.

acs_demo_subset=acs_demo[['GEOID20','population' ]]

# Making sure that the crs- coordinate reference system for census_2020 is the same as that of the la_nc using the to_crs()
# method. This will allow the spatial merging of both the geopandas dataframes.

census_2020 = census_2020.to_crs(la_nc.crs)

# Spatial overlap of the la_nc and census_2020 data.
census_NC = gpd.overlay(census_2020, la_nc, how='intersection')

census_NC= census_NC[['TRACTCE20','GEOID20', 'NAME', 'NC_ID', 'INTPTLAT20', 'INTPTLON20', 'geometry']]
print('Number of rows in census_NC: ', census_NC.shape[0])  

Number of rows in census_NC:  2108


In [5]:
# Let us take a look at the spatially merged data:
census_NC.head()

,TRACTCE20,GEOID20,NAME,NC_ID,INTPTLAT20,INTPTLON20,geometry
0,101110,06037101110,SUNLAND-TUJUNGA NC,10,+34.2594737,-118.2929869,"POLYGON ((-118.30229 34.25870, -118.30091 34.2..."
1,101122,06037101122,SUNLAND-TUJUNGA NC,10,+34.2677213,-118.2901465,"POLYGON ((-118.30334 34.27371, -118.30330 34.2..."
2,101220,06037101220,SUNLAND-TUJUNGA NC,10,+34.2516083,-118.2816328,"POLYGON ((-118.28592 34.25227, -118.28592 34.2..."
3,101221,06037101221,SUNLAND-TUJUNGA NC,10,+34.2543290,-118.2925767,"POLYGON ((-118.29945 34.25598, -118.29792 34.2..."
4,101222,06037101222,SUNLAND-TUJUNGA NC,10,+34.2513519,-118.2885261,"POLYGON ((-118.29434 34.25233, -118.29318 34.2..."


In [6]:
# Merging the census_NC and acs_demo_subset.
census = pd.merge(census_NC, acs_demo_subset, on='GEOID20')

### Displaying the duplicate entries- census tracts intersecting more than 1 Neighborhood councils

In [7]:
df_duplicate= census_NC[census_NC.GEOID20.duplicated(keep=False)]
df_duplicate.sort_values(by=['GEOID20']).head()

,TRACTCE20,GEOID20,NAME,NC_ID,INTPTLAT20,INTPTLON20,geometry
1,101122,06037101122,SUNLAND-TUJUNGA NC,10,+34.2677213,-118.2901465,"POLYGON ((-118.30334 34.27371, -118.30330 34.2..."
17,101122,06037101122,FOOTHILL TRAILS DISTRICT NC,9,+34.2677213,-118.2901465,"POLYGON ((-118.29785 34.27778, -118.29783 34.2..."
5,101300,06037101300,SUNLAND-TUJUNGA NC,10,+34.2487777,-118.2709990,"POLYGON ((-118.27822 34.25068, -118.27822 34.2..."
18,101300,06037101300,FOOTHILL TRAILS DISTRICT NC,9,+34.2487777,-118.2709990,"POLYGON ((-118.26682 34.23124, -118.26695 34.2..."
6,101400,06037101400,SUNLAND-TUJUNGA NC,10,+34.2428521,-118.2941612,"POLYGON ((-118.32227 34.24961, -118.32212 34.2..."


### Before moving forward, for sanity check, let us try to get the population of distinct census tracts without worrying about the proportion of NC areas intersected by census tracts. To do this, I will retain the first row of the  duplicate data for the entire dataframe.

In [8]:
test =  census.groupby('TRACTCE20').first()
test.population.sum()

4534712

<div style="text-align: justify">  
    
### This number exactly matches with the one that I get using this [notebook](https://github.com/priyakalyan/Updated_NC_pop/blob/main/NC_pop_recent_no_filter.ipynb) where the population of NCs have been calculated at the tract level. This implies that the census tracts that is being used in the calculation is wrong. This issue will be addressed here. But before going ahead, the next section will outline the methodology to calculate the area of census tracts intersecting multiple NCs.

### So here is a method to take care of cases where the census tract intersects more than 1 NCs. 

- **Find the area for each entry in census dataframe.** 
- **Group the census dataframe by 'TRACTCE20' and then find sum of the area- this gives you the total area of each census       tract.** 
- **Next, find the percentage of area of the census tract intersecting different NCs.** 
- **Use this information to find the percentage of population for each census tract.**
- **Then find the total population by grouping them by NCs.** 

In [9]:
# Add the area column.
census = pd.merge(census_NC, acs_demo_subset, on='GEOID20')
census['area(sq_miles)']= census.apply(lambda x: area_sq_miles(x.geometry), axis=1)
census[['NC_ID','NAME', 'TRACTCE20', 'population','area(sq_miles)']].head(10)

,NC_ID,NAME,TRACTCE20,population,area(sq_miles)
0,10,SUNLAND-TUJUNGA NC,101110,3923,0.441083
1,10,SUNLAND-TUJUNGA NC,101122,4119,1.020872
2,9,FOOTHILL TRAILS DISTRICT NC,101122,4119,0.000038
3,10,SUNLAND-TUJUNGA NC,101220,3775,0.269841
4,10,SUNLAND-TUJUNGA NC,101221,3787,0.136748
5,10,SUNLAND-TUJUNGA NC,101222,2717,0.114484
6,10,SUNLAND-TUJUNGA NC,101300,3741,0.993003
7,9,FOOTHILL TRAILS DISTRICT NC,101300,3741,0.002387
8,10,SUNLAND-TUJUNGA NC,101400,3246,2.414663
9,9,FOOTHILL TRAILS DISTRICT NC,101400,3246,0.021664


In [10]:
census_test=census.groupby('TRACTCE20', as_index= False).agg({'NC_ID':'first','NAME':'first','geometry':'first', 'INTPTLAT20':'first', 'INTPTLON20':'first', 'NAME':'first','GEOID20':'first','area(sq_miles)':sum})
census_test.rename(columns={'area(sq_miles)': 'total_area(sq_miles)'}, inplace=True)
census_test= census_test[['TRACTCE20','total_area(sq_miles)']]

In [11]:
# Let us add the total_area to census dataframe.
census_perc= pd.merge(census, census_test, on='TRACTCE20')
census_perc.head()

,TRACTCE20,GEOID20,NAME,NC_ID,INTPTLAT20,INTPTLON20,geometry,population,area(sq_miles),total_area(sq_miles)
0,101110,06037101110,SUNLAND-TUJUNGA NC,10,+34.2594737,-118.2929869,"POLYGON ((-118.30229 34.25870, -118.30091 34.2...",3923,0.441083,0.441083
1,101122,06037101122,SUNLAND-TUJUNGA NC,10,+34.2677213,-118.2901465,"POLYGON ((-118.30334 34.27371, -118.30330 34.2...",4119,1.020872,1.020910
2,101122,06037101122,FOOTHILL TRAILS DISTRICT NC,9,+34.2677213,-118.2901465,"POLYGON ((-118.29785 34.27778, -118.29783 34.2...",4119,0.000038,1.020910
3,101220,06037101220,SUNLAND-TUJUNGA NC,10,+34.2516083,-118.2816328,"POLYGON ((-118.28592 34.25227, -118.28592 34.2...",3775,0.269841,0.269841
4,101221,06037101221,SUNLAND-TUJUNGA NC,10,+34.2543290,-118.2925767,"POLYGON ((-118.29945 34.25598, -118.29792 34.2...",3787,0.136748,0.136748


In [12]:
# Evaluating the percentage of the intersecting areas (area_perc):
census_perc['area_perc'] = census_perc['area(sq_miles)']/census_perc['total_area(sq_miles)']

# Adding the percentage of population column
census_perc['total_population'] = census_perc['population']*census_perc['area_perc']   

<div style="text-align: justify">  
    
### I will be incorporating the area filter and the population filter to skim out the values that were  population of the NCs. 

### Let us take a look at the NC- Atwater Village. 

In [13]:
census_perc[census_perc['NAME'] == 'ATWATER VILLAGE NC'][['TRACTCE20', 'NAME', 'NC_ID', 'population', 'area(sq_miles)', 'area_perc', 'total_area(sq_miles)', 'total_population']]

,TRACTCE20,NAME,NC_ID,population,area(sq_miles),area_perc,total_area(sq_miles),total_population
811,189703,ATWATER VILLAGE NC,37,1880,0.043906,0.029314,1.497769,55.110346
833,311601,ATWATER VILLAGE NC,37,2608,0.000209,0.149092,0.001404,388.832888
837,311700,ATWATER VILLAGE NC,37,5966,0.004878,0.322874,0.015109,1926.264006
841,980009,ATWATER VILLAGE NC,37,5,6.816197,0.312739,21.795157,1.563695
973,187102,ATWATER VILLAGE NC,37,4376,0.340444,0.402493,0.845840,1761.307749
984,187101,ATWATER VILLAGE NC,37,3213,0.302714,0.995004,0.304234,3196.947376
986,187200,ATWATER VILLAGE NC,37,3324,0.011649,0.031713,0.367315,105.412983
990,187300,ATWATER VILLAGE NC,37,3715,0.004740,0.010301,0.460138,38.266922
994,188100,ATWATER VILLAGE NC,37,3960,0.813095,1.000000,0.813095,3960.000000
995,188201,ATWATER VILLAGE NC,37,3262,0.019628,0.058066,0.338025,189.409874


### Look at the census tracts 301601, 301602, 301702, 311801. When I looked at the map, these were the tracts belonging to the neighboring city- Glendale. These tracts obviously do not belong to Atwater Village NC. The total area of these tracts are extremely small, but the area_perc for some of these tracts - 1 and so the whole population is taken into account and thus this will result in a higher population value than the expected one. So, I am adding the area filter- total_area(sq_miles) >= 0.01. 
    
### Similarly, some of the population value is so small, but the cumulative effect can result in an exaggerated population number. Therefore, I am going to add the population filter here. The [minimum population](https://www2.census.gov/geo/pdfs/education/CensusTracts.pdf) of a whole census tract is 1200. Since we are dealing with proportional areas and sometimes, a census tract intersects 2 NCs in 30% to 70% ratio and so I want to be reasonable with the population filter- 1000 (we do not want to undercount too). 


In [14]:
# Filtering out total_area < 0.01 square miles.
census_perc_filter = census_perc[census_perc['total_area(sq_miles)'] >= 0.01]
census_perc_filter.head()

,TRACTCE20,GEOID20,NAME,NC_ID,INTPTLAT20,INTPTLON20,geometry,population,area(sq_miles),total_area(sq_miles),area_perc,total_population
0,101110,06037101110,SUNLAND-TUJUNGA NC,10,+34.2594737,-118.2929869,"POLYGON ((-118.30229 34.25870, -118.30091 34.2...",3923,0.441083,0.441083,1.000000,3923.000000
1,101122,06037101122,SUNLAND-TUJUNGA NC,10,+34.2677213,-118.2901465,"POLYGON ((-118.30334 34.27371, -118.30330 34.2...",4119,1.020872,1.020910,0.999963,4118.846169
2,101122,06037101122,FOOTHILL TRAILS DISTRICT NC,9,+34.2677213,-118.2901465,"POLYGON ((-118.29785 34.27778, -118.29783 34.2...",4119,0.000038,1.020910,0.000037,0.153831
3,101220,06037101220,SUNLAND-TUJUNGA NC,10,+34.2516083,-118.2816328,"POLYGON ((-118.28592 34.25227, -118.28592 34.2...",3775,0.269841,0.269841,1.000000,3775.000000
4,101221,06037101221,SUNLAND-TUJUNGA NC,10,+34.2543290,-118.2925767,"POLYGON ((-118.29945 34.25598, -118.29792 34.2...",3787,0.136748,0.136748,1.000000,3787.000000


In [15]:
# Removing the rows with population value < 500.
census_filter = census_perc_filter[census_perc_filter['total_population'] >= 1000]
census_filter.shape[0]

1172

In [16]:
# Summming up the population of each NCs.
census_pop =census_filter.groupby('NAME', as_index= False).agg({'NC_ID':'first',  'NAME':'first', 'total_population' : sum})
census_pop.head(15)

,NC_ID,NAME,total_population
0,6,ARLETA NC,35588.766140
1,42,ARROYO SECO NC,19483.632934
2,46,ARTS DISTRICT LITTLE TOKYO NC,4153.255598
3,37,ATWATER VILLAGE NC,14666.519132
4,64,BEL AIR-BEVERLY CREST NC,24736.886644
5,50,BOYLE HEIGHTS NC,87820.011105
6,13,CANOGA PARK NC,54188.056262
7,110,CENTRAL ALAMEDA NC,30220.267167
8,32,CENTRAL HOLLYWOOD NC,18207.193924
9,95,CENTRAL SAN PEDRO NC,29246.324351


In [17]:
census_pop.total_population.sum()

3826003.3939952855

<div style="text-align: justify">  
    
### This number is close to the [Census Bureau](https://www.census.gov/quickfacts/losangelescitycalifornia?) value- 3,849,297. Now that we have the population of each NC, let us move on to getting the area and finally the updated population density of the neighborhood councils.      

In [18]:
# Grouping the original census dataframe by NAME and then summing up the total area- this gives the area of each neighborhood 
# council. Very important note here- groupby function works when trying to aggregate dataframes but for spatial data, we can
# aggregate the geometry features using dissolve function.

census_area = census.dissolve(by= 'NAME', as_index= False, aggfunc=({'NC_ID':'first','area(sq_miles)' : sum }))

# Gathering all the columns of interest. 
census_final = census_area.join(census_pop['total_population'])
census_final['pop_density']= census_final['total_population']/census_final['area(sq_miles)']

# Rearranging the columns.
census_final = census_final[['NAME', 'geometry', 'NC_ID', 'total_population', 'area(sq_miles)', 'pop_density']]
census_final.head(15)

,NAME,geometry,NC_ID,total_population,area(sq_miles),pop_density
0,ARLETA NC,"POLYGON ((-118.41010 34.23309, -118.41034 34.2...",6,35588.766140,3.284868,10834.154234
1,ARROYO SECO NC,"POLYGON ((-118.18576 34.09293, -118.18576 34.0...",42,19483.632934,3.063327,6360.285727
2,ARTS DISTRICT LITTLE TOKYO NC,"POLYGON ((-118.22877 34.04155, -118.22827 34.0...",46,4153.255598,0.879216,4723.819709
3,ATWATER VILLAGE NC,"POLYGON ((-118.25399 34.10816, -118.25424 34.1...",37,14666.519132,8.748450,1676.470626
4,BEL AIR-BEVERLY CREST NC,"POLYGON ((-118.46573 34.07325, -118.46581 34.0...",64,24736.886644,17.038756,1451.801231
5,BOYLE HEIGHTS NC,"POLYGON ((-118.20504 34.01263, -118.20504 34.0...",50,87820.011105,5.735881,15310.641644
6,CANOGA PARK NC,"POLYGON ((-118.58846 34.19524, -118.58846 34.1...",13,54188.056262,3.689892,14685.540068
7,CENTRAL ALAMEDA NC,"POLYGON ((-118.23777 33.98933, -118.23777 33.9...",110,30220.267167,1.358014,22253.279485
8,CENTRAL HOLLYWOOD NC,"POLYGON ((-118.32445 34.08712, -118.32445 34.0...",32,18207.193924,1.229127,14813.116239
9,CENTRAL SAN PEDRO NC,"POLYGON ((-118.28794 33.73151, -118.28795 33.7...",95,29246.324351,2.438025,11995.910754


### I am going to go ahead and plot the census_final using geopandas explore- folium.

In [19]:
m= census_final.explore(
    column= 'NAME', # make choropleth based on 'NC name' column
    name='NC Regions', 
    tooltip='NAME', # show 'NC name' value in tooltip (on hover)
    color="red", # use red color on all points
    popup=True, # show all values in popup (on click)
    tiles="openstreetmap", # use "openstreetmap" tiles
    cmap="Set1", # use "Set1" matplotlib colormap
    style_kwds=dict(color="black"), # use black outline
    legend=False
     )
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)
folium.TileLayer('Stamen Water Color').add_to(m)
folium.TileLayer('cartodbpositron').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)
folium.LayerControl().add_to(m)
m.save('census_final.html')
webbrowser.open('census_final.html')

True

In [20]:
# Save the dataframe as a csv file after dropping the geometry column.
census_final= census_final.drop(['geometry'], axis=1)
census_final.to_csv('census_final.csv')